# 허깅페이스로 나만의 BERT Pretraining 해보기 튜토리얼 자료

**진행자:** [윤주성](https://www.linkedin.com/in/joosung-yoon/)<br>
**참고자료:** [Pretraining BERT with Hugging Face Transformers](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/pretraining_BERT.ipynb)

## Introduction
### BERT (Bidirectional Encoder Representations from Transformers)
컴퓨터비전 분야에서는 ImageNet 데이터로 학습한 모델을 transfer learning(finetuning)하면 모델 성능을 높이는데 뛰어난 효과가 있다고 알려져 있었습니다. 최근 몇 년 동안 자연어처리 분야에서도 이와 비슷한 테크닉을 적용했을때 모델 성능이 크게 개선된다는 것이 밝혀졌습니다. Google에서 발표한 BERT 모델은 단어 사이의 문맥 관계를 attention mechanism을 통해 학습하는 Transformer 기반 모델로 자연어처리 분야에서의 대표적인 Pretraining 모델입니다. 기존 Transformer 구조에서 encoder 부분만을 활용해서 Language model을 학습한 모델입니다. 자세한 내용은 논문을 참조하시기 바랍니다.

언어 모델을 학습할때 가장 중요한건 어떤 방법으로 모델을 학습할지 정하는 것입니다. 대부분의 모델은 다음에 나올 단어를 예측합니다 (e.g. `"버트를 이용한 언어모델 _"`),
이러한 방법은 간단하지만 context를 한 방향으로만 활용할 수 있다는 한계점이 있습니다. BERT는 이러한 한계점을 극복하기 위해 다음의 두가지 방법을 제안했습니다.

### Masked Language Modeling (MLM)

입력 시퀀스의 15%를 `[MASK]` token으로 변경하고 이 `[MASK]` 토큰 자리에 있던 원래 단어를 예측합니다.

### Next Sentence Prediction (NSP)

BERT에서는 입력을 pair로 받아서 학습합니다. 입력으로 사용되는 pair의 50%는 같은 문서에서 나머지 50%는 다른 랜덤한 문서에서 가져옵니다. NSP는 pair로 입력된 시퀀스가 같은 문서에서 온 것인지 다른 문서에서 온 것인지를 예측하는 sequence-level task입니다.

학습에 사용할 한국어 dataset 허깅페이스 🤗 Datasets을 통해 사용할 수 있습니다.

**시작하기 전에 주의할 점**: Colab에서 실습하시게 되면 세션 설정에 반드시 GPU runtime을 설정해주시기 바랍니다.

## SetUp
### Installing the requirements

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install soynlp 
!pip install emoji==1.7.0
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-9tdqpdhw
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-9tdqpdhw
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached emoji-1.7.0-py3-none-any.whl
  Attempting uninstall: emoji
    Found 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached emoji-1.2.0-py3-none-any.whl (131 kB)
  Attempting uninstall: emoji
    Found existing installation: emoji 1.7.0
    Uninstalling emoji-1.7.0:
      Successfully uninstalled emoji-1.7.0


### Importing the necessary libraries

In [ ]:
import nltk
import random
import logging
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from kss import split_sentences

# nltk.download("punkt")
# 에러 메세지만 로깅합니다
tf.get_logger().setLevel(logging.ERROR)
# 랜덤 시드를 설정합니다
tf.keras.utils.set_random_seed(42)

### Define certain variables

In [ ]:
TOKENIZER_BATCH_SIZE = 256  # 토크나이저를 학습할 때 사용할 배치 사이즈
TOKENIZER_VOCABULARY = 25000  # 토크나이저의 vocab 사이즈

BLOCK_SIZE = 128  # input sample에 있는 token의 최대 수 
NSP_PROB = 0.50  # 다음 문장이 같은 문서에 있을 NSP 확률
SHORT_SEQ_PROB = 0.1  # pretraining과 finetuning시 차이를 줄이기 위한 짧은 문장 생성 확률


MLM_PROB = 0.15  # 마스킹 MLM 확률 

TRAIN_BATCH_SIZE = 2  # 모델 배치 사이즈
MAX_EPOCHS = 1  # 모델 최대 에폭
LEARNING_RATE = 1e-4  # Learning rate

MODEL_CHECKPOINT = "klue/bert-base" # 🤗 Model Hub에 올라와 있는 모델명
MAX_LENGTH = 512  # input sample을 패딩 처리한 후 토큰의 최대 수

## Load the Wiki dataset
Wikipedia 데이터셋은 언어 모델에서 많이 사용되는 데이터셋입니다.  [🤗 허깅페이스의 Datasets](https://github.com/huggingface/datasets) 라이브러리의 load_dataset함수를 통해 데이터를 로딩합니다

In [ ]:
from datasets import load_dataset

dataset = load_dataset("lcw99/wikipedia-korean-20221001", split="train") 

In [ ]:
print(dataset)

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 607256
})


In [ ]:
TRAIN_TEST_SPLIT = 0.1

dataset = dataset.train_test_split(
    train_size=1-TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)
dataset['validation'] = dataset['test']
del dataset['test']
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 546530
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 60726
    })
})


In [ ]:
dataset['train'][0]

{'id': '1773240',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%84%EC%B4%8C%EB%A6%AC',
 'title': '진촌리',
 'text': '진촌리는 다음 지역에 위치한 대한민국의 리이다.\n\n 인천광역시 옹진군 백령면 진촌리(鎭村里)\n 경기도 안성시 미양면 진촌리(眞村里)\n 경기도 안성시 삼죽면 진촌리(眞村里)\n\n같이 보기 \n\n대한민국의 리'}

## Training a new Tokenizer
나만의 언어 모델을 만들기전에 주어진 corpus로부터 나만의 tokenizer를 학습 할수 있습니다.  
Transformer 모델들은 대부분 subword tokenization algorithms을 사용하기 때문에 우리가 사용하는 corpus에 맞는 tokenizer를 학습하는 것이 필요할 수 있습니다.  
🤗 Transformers 라이브러리에 있는 `Tokenizer`를 통해 학습을 진행합니다.  
먼저 Wiki corpus dataset으로부터 raw document를 가져옵니다.

In [ ]:
all_texts = [
    doc for doc in dataset["train"]["text"] if len(doc) > 0
]

`batch_iterator` 를 통해 tokenizer 학습에 필요한 데이터를 전달해줍니다.  
`VOCAB_TRAIN_DOC_SIZE`는 tokenizer를 학습할 corpus 규모에 따라 설정할 수 있습니다.  
실습 진행을 위해 5000개의 문서만 사용하겠습니다.

In [ ]:
# tokenizer 학습셋 구축
VOCAB_TRAIN_DOC_SIZE = 5000

def batch_iterator():
    for i in tqdm(range(0, VOCAB_TRAIN_DOC_SIZE, TOKENIZER_BATCH_SIZE)):
        yield all_texts[i : i + TOKENIZER_BATCH_SIZE]

이 예제에서는 기존에 공개된 알고리즘과 파라미터를 공유하는 `klue/bert-base` 모델의 토크나이저를 가져와서 재학습하고자 합니다.  
이를 위해 먼저 tokenizer를 로딩합니다.

In [ ]:
from transformers import AutoTokenizer
print(f"tokenizer from: {MODEL_CHECKPOINT}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

tokenizer from: klue/bert-base


이제 tokenizer를 Wiki 데이터셋에 대해서 재학습합니다.

In [ ]:
tokenizer = tokenizer.train_new_from_iterator(
    batch_iterator(), vocab_size=TOKENIZER_VOCABULARY
)

100%|██████████| 20/20 [00:00<00:00, 23.57it/s]


재학습이 끝난 후 나만의 새로운 토크나이저를 얻게 되었습니다. 다음 전처리 스텝으로 이동해보겠습니다.

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='klue/bert-base', vocab_size=25000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

## Data Pre-processing
전체적인 학습과정을 실습하기 위해 데이터셋의 일부만 샘플링해서 학습에 적용하겠습니다.  
beomi님의 kcbert에서 사용하는 한글 전처리 로직을 통해 corpus를 전체적으로 전처리하겠습니다.

In [ ]:
DOC_NUM = 500
dataset["train"] = dataset["train"].select([i for i in range(DOC_NUM)])
dataset["validation"] = dataset["validation"].select([i for i in range(DOC_NUM)])

In [ ]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = list({y for x in emoji.UNICODE_EMOJI.values() for y in x.keys()})
emojis = ''.join(emojis)
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

# 학습데이터셋 전처리 함수
def clean(ds):
    """
    ref: https://huggingface.co/beomi/kcbert-base
    """
    x = ds['text']
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    ds['text'] = x
    return ds


In [ ]:
dataset = dataset.map(clean)

In [ ]:
dataset['train'][0]

{'id': '1773240',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%84%EC%B4%8C%EB%A6%AC',
 'title': '진촌리',
 'text': '진촌리는 다음 지역에 위치한 대한민국의 리이다. 인천광역시 옹진군 백령면 진촌리( ) 경기도 안성시 미양면 진촌리( ) 경기도 안성시 삼죽면 진촌리( ) 같이 보기 대한민국의 리'}

BERT는 학습은 크게 MLM, NSP task로 구성되어있습니다.  
MLM task는 Transformers에서 `DataCollatorForLanguageModeling`를 통해 쉽게 구현할 수 있지만 `NSP`의 경우 직접 작업이 필요합니다.   
`prepare_train_features` 함수를 통해 `NSP`task를 위한 sentence pair (A, B) 를 구성해보겠습니다.

문장 분리를 위해 영어는 `nltk.tokenize.sent_tokenize`, 한국어는  `kss.split_sentences` 함수를 사용하겠습니다.

In [ ]:
# 학습 샘플에 대한 max_num_tokens 정의
max_num_tokens = BLOCK_SIZE - tokenizer.num_special_tokens_to_add(pair=True)
print(f"max_num_tokens:{max_num_tokens}")

def prepare_train_features(examples):

    """NSP task를 위한 Function 

    Arguments:
      examples: A dictionary with 1 key ("text")
        text: List of raw documents (str)
    Returns:
      examples:  A dictionary with 4 keys
        input_ids: List of tokenized, concatnated, and batched
          sentences from the individual raw documents (int)
        token_type_ids: List of integers (0 or 1) corresponding
          to: 0 for senetence no. 1 and padding, 1 for sentence
          no. 2
        attention_mask: List of integers (0 or 1) corresponding
          to: 1 for non-padded tokens, 0 for padded
        next_sentence_label: List of integers (0 or 1) corresponding
          to: 1 if the second sentence actually follows the first,
          0 if the senetence is sampled from somewhere else in the corpus
    """

    # 학습에 사용할 유효 데이터셋 필터링
    examples["document"] = [
        d.strip() for d in examples["text"] if len(d) > 0
    ]
    # 문서를 문장 단위로 쪼개기
    examples["sentences"] = [
        # nltk.tokenize.sent_tokenize(document) for document in examples["document"]
        split_sentences(document) for document in examples["document"]        
    ]
    # 학습한 tokenizer로 token -> token_id로 변환
    examples["tokenized_sentences"] = [
        [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)) for sent in doc]
        for doc in examples["sentences"]
    ]

    # 출력 변수 정의
    examples["input_ids"] = []
    examples["token_type_ids"] = []
    examples["attention_mask"] = []
    examples["next_sentence_label"] = []

    for doc_index, document in enumerate(examples["tokenized_sentences"]):
        
        current_chunk = []  # 현재 처리하고 있는 segment를 저장하는 chunk
        current_length = 0
        i = 0

        # 보통의 경우 short sequence는 학습에 비효율적이지만, pretraining과 finetuning의 갭을 줄이기 위해 
        # 의도적으로 SHORT_SEQ_PROB(10%)만큼 short seq를 추가해줍니다
        target_seq_length = max_num_tokens

        if random.random() < SHORT_SEQ_PROB:
            target_seq_length = random.randint(2, max_num_tokens) # 2~max_num_tokens 사이의 random length 생성

        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)

            # 처음에 로직에 진입할 조건 or max token수보다 길이가 긴 경우 진입
            if i == len(document) - 1 or current_length >= target_seq_length:
                if current_chunk:
                    # `a_end` 변수는 `current_chunk`에 있는 세그먼트중 몇개가 first sentence에 들어갈지를 나타냄
                    a_end = 1

                    # chunk가 2개 이상이면 랜덤하게 1개 이상의 chunk 개수를 선택 후 
                    # first sentence에 추가해줘서 target_seq_length를 넘지 않는 first sentence를 만듦
                    if len(current_chunk) >= 2:
                        a_end = random.randint(1, len(current_chunk) - 1)

                    # tokens_a에 current_chunk를 저장
                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])

                    tokens_b = []

                    if len(current_chunk) == 1 or random.random() < NSP_PROB:
                        # 랜덤 문서 선택하는 케이스
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        # 랜덤하게 선택한 문서가 기존과 같은 문서인지 체크
                        for _ in range(10):
                            random_document_index = random.randint(
                                0, len(examples["tokenized_sentences"]) - 1
                            )
                            if random_document_index != doc_index:
                                break

                        # 랜덤 문서 추출
                        random_document = examples["tokenized_sentences"][
                            random_document_index
                        ]

                        # 랜덤 문서 내에서 임의의 시작점 추출
                        random_start = random.randint(0, len(random_document) - 1)

                        # 랜덤 문서에서 임의의 토큰들 추출, target_b_length에 도달하면 스탑
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        
                        # 사용하지 않은 segment만큼 다시 i 를 복원해줌
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    else:
                        # 같은 문서 선택하는 케이스
                        is_random_next = False

                        # 현재 문서에서 먼저 추출된 토큰 뒷부분 부터 추출
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    input_ids = tokenizer.build_inputs_with_special_tokens(
                        tokens_a, tokens_b
                    )
                    # token type ids를 추가함. sentence A에는 0을, sentence B에는 1을 값으로 줌
                    token_type_ids = tokenizer.create_token_type_ids_from_sequences(
                        tokens_a, tokens_b
                    )

                    padded = tokenizer.pad(
                        {"input_ids": input_ids, "token_type_ids": token_type_ids},
                        padding="max_length",
                        max_length=MAX_LENGTH
                    )                    
                    
                    # MAX_LENGTH 이하의 문서만 남도록 길이 수정
                    for k, v in padded.items():
                      padded[k] = v[:MAX_LENGTH]


                    examples["input_ids"].append(padded["input_ids"])
                    examples["token_type_ids"].append(padded["token_type_ids"])
                    examples["attention_mask"].append(padded["attention_mask"])
                    examples["next_sentence_label"].append(1 if is_random_next else 0)
                    
                    current_chunk = []
                    current_length = 0
            i += 1

    # 데이터셋에서 사용하지 않는 column 제거
    del examples["document"]
    del examples["sentences"]
    del examples["text"]
    del examples["tokenized_sentences"]

    return examples


tokenized_dataset = dataset.map(
    prepare_train_features, batched=True, remove_columns=['id', 'url', 'title', 'text'], num_proc=1,
)

max_num_tokens:125


  0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (860 > 512). Running this sequence through the model will result in indexing errors
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/1 [00:00<?, ?ba/s]

MLM task는 입력 토큰의 일부를 `[MASK]`로 교체하는데, `DataCollatorForLanguageModeling`를 통해 쉽게 구현 할 수 있습니다.

In [ ]:
from transformers import DataCollatorForLanguageModeling

collater = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=MLM_PROB, return_tensors="tf"
)

In [ ]:
train = tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["labels", "next_sentence_label"],
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    collate_fn=collater,
)

validation = tokenized_dataset["validation"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["labels", "next_sentence_label"],
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    collate_fn=collater,
)

## Defining the model
모델을 정의하기 위해 어떤 모델에 대한 config를 쓸 것인지에 대한 정보가 필요합니다.  
본 실습에서는 `klue/bert-base` 모델에 대한 config 정보를 사용합니다.

In [ ]:
from transformers import BertConfig

config = BertConfig.from_pretrained(MODEL_CHECKPOINT)

모델 학습을 위해 🤗Transformers 라이브러리의 `TFBertForPreTraining` 클래스를 사용합니다.

In [ ]:
from transformers import TFBertForPreTraining

model = TFBertForPreTraining(config)

optimizer를 정의하고 모델을 compile해줍니다.  
이때 loss에 대한 계산은 내부적으로 이뤄집니다. 

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


셋팅이 완료되었으므로 모델 학습을 시작해보겠습니다.

In [ ]:
%%time
model.fit(train, validation_data=validation, epochs=MAX_EPOCHS)

1000/1000 [==============================] - 443s 427ms/step - loss: 9.5061 - val_loss: 9.4193
CPU times: user 4min 52s, sys: 1min, total: 5min 52s
Wall time: 7min 22s


학습한 모델은 다음과 같이 로컬에 저장할 수 있습니다.

In [ ]:
model.save_pretrained("keras-bert")

In [ ]:
!ls keras-bert

config.json  tf_model.h5


In [ ]:
from transformers import pipeline

# fill_mask = pipeline("fill-mask", 
#                      model='./keras-bert', 
#                      tokenizer=tokenizer, 
#                      framework="tf")

fill_mask = pipeline("fill-mask", 
                     model="klue/bert-base")

fill_mask("대한민국의 수도는 [MASK]입니다")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.448245108127594,
  'token': 3671,
  'token_str': '서울',
  'sequence': '대한민국의 수도는 서울 입니다'},
 {'score': 0.08845192939043045,
  'token': 9474,
  'token_str': '광화문',
  'sequence': '대한민국의 수도는 광화문 입니다'},
 {'score': 0.06651604175567627,
  'token': 3902,
  'token_str': '부산',
  'sequence': '대한민국의 수도는 부산 입니다'},
 {'score': 0.04384538158774376,
  'token': 7141,
  'token_str': '평양',
  'sequence': '대한민국의 수도는 평양 입니다'},
 {'score': 0.03615731745958328,
  'token': 4431,
  'token_str': '대전',
  'sequence': '대한민국의 수도는 대전 입니다'}]

학습한 모델은 다음과 같은 model명(e.g. `"your-username/the-name-you-picked"`)을 설정 후 Hugging Face Model Hub에 올릴 수 있습니다.



```python
model.push_to_hub("pretrained-bert", organization="keras-io")
tokenizer.push_to_hub("pretrained-bert", organization="keras-io")
```

모델이 model hub에 푸시되면 아래와 같은 방법으로 로딩할 수 있습니다.

```python
from transformers import TFBertForPreTraining

model = TFBertForPreTraining.from_pretrained("your-username/my-awesome-model")
```
finetuning 할 때는 아래와 같이 로딩이 가능합니다.

```python
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained("your-username/my-awesome-model")
```
이 경우에는, pretraining head가 로딩되지 않고 새로운 task에 맞는 head가 랜덤하게 초기화되서 추가됩니다.
